# Exploratory Data Analysis Allobank FAQ

## Import Library

In [8]:
# Import Library
import pandas as pd
import numpy as np
import matplotlib as plt
import re
from wordcloud import WordCloud
from sklearn.feature_extraction.text import CountVectorizer

## Load Data FAQ

In [9]:
# Load data FAQ
df = pd.read_csv('../Dataset/faq_allobank.csv').copy()
df.head()

,category,question,answer
0,UMUM,Apa itu Allo Bank?,Sebuah layanan perbankan berbasis digital yang...
1,UMUM,Kapan Allo Bank didirikan di Indonesia?,Allo Bank didirikan di Indonesia pada tahun 2021.
2,UMUM,Apakah Allo Bank terdaftar dan diawasi oleh Ot...,Benar! Allo Bank telah terdaftar dan diawasi o...
3,UMUM,Apakah menyimpan dana di Allo Bank dipastikan ...,Tentu saja! Allo Bank terdaftar sebagai pesert...
4,UMUM,Apakah Allo Bank memiliki kantor cabang?,"Saat ini, Allo Bank memiliki dua kantor cabang..."


## Data Analysis

In [10]:
# Display Column Information
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 369 entries, 0 to 368
Data columns (total 3 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   category  369 non-null    object
 1   question  369 non-null    object
 2   answer    369 non-null    object
dtypes: object(3)
memory usage: 8.8+ KB


In [11]:
# Add column character length for analysis
df["question_len"] = df["question"].apply(len)
df["answer_len"] = df["answer"].apply(len)

In [12]:
# Statistic Overview
df.describe()

,question_len,answer_len
count,369.000000,369.000000
mean,68.864499,246.701897
std,24.674139,222.886262
min,13.000000,49.000000
25%,53.000000,110.000000
50%,66.000000,189.000000
75%,83.000000,296.000000
max,172.000000,2438.000000


In [17]:
# Data Duplicates Check
question_duplicates = df.duplicated("question").sum()
print(f"Duplikat pertanyaan: {question_duplicates}")

Duplikat pertanyaan: 12


In [18]:
df[df.duplicated("question")]

,category,question,answer,question_len,answer_len
65,AKUN,Di mana saya bisa menemukan file e-statement s...,Anda bisa mendapatkan file e-statement pada ta...,50,102
140,LAYANAN,Apa itu Allo Pay?,Allo Pay adalah produk uang elektronik jenis u...,17,234
154,LAYANAN,Bagaimana jika KTP saya sudah menggunakan KTP ...,Saat ini Allo Bank belum mendukung registrasi ...,54,111
211,LAYANAN,How much does it cost to save money in the All...,There are no administration fees to customers ...,53,76
222,LAYANAN,Bagaimana caranya jika saya ingin membatalkan ...,Setelah Anda mengkonfirmasi persetujuan pengaj...,56,231
227,LAYANAN,Jika tanggal jatuh tempo jatuh pada hari libur...,"Ya, Anda tetap bisa melakukan pembayaran meski...",87,92
228,LAYANAN,Apakah saya akan mendapatkan notifikasi pada s...,Anda akan mendapatkan notifikasi sejak H-1 dan...,80,126
229,LAYANAN,Apakah saya dapat mengajukan lebih dari 1 (sat...,"Ya, Anda dapat melakukan pengajuan lebih dari ...",58,103
301,LAYANAN,Bagaimana jika saya salah memasukan nomor ID p...,"Mohon maaf, Anda tidak dapat melakukan penggan...",100,223
303,LAYANAN,Jumlah token yang masuk tidak sesuai dengan pe...,"Tidak perlu khawatir, Anda dapat menghubungi A...",83,296


In [14]:
answer_duplicates = df.duplicated(subset=["answer"]).sum()
print(f"Duplikat jawaban: {answer_duplicates}")

Duplikat jawaban: 30


In [19]:
df[df.duplicated("answer")]

,category,question,answer,question_len,answer_len
65,AKUN,Di mana saya bisa menemukan file e-statement s...,Anda bisa mendapatkan file e-statement pada ta...,50,102
72,AKUN,Apa yang harus saya lakukan ketika saya tetap ...,"Tidak perlu khawatir, Anda dapat menghubungi A...",109,267
88,AKUN,Apakah saya bisa mengajukan pembukaan blokir a...,"Tidak perlu khawatir, Anda dapat menghubungi A...",68,267
89,AKUN,Apakah ada syarat dan ketentuan untuk mengajuk...,"Tidak perlu khawatir, Anda dapat menghubungi A...",71,267
90,AKUN,Apa saja dokumen pendukung yang dibutuhkan unt...,"Tidak perlu khawatir, Anda dapat menghubungi A...",71,267
97,PRODUK,Mengapa saya tidak bisa menggunakan Allo Pay+ ...,Mungkin Kamu telah melebihi batas pembayaran p...,51,75
140,LAYANAN,Apa itu Allo Pay?,Allo Pay adalah produk uang elektronik jenis u...,17,234
154,LAYANAN,Bagaimana jika KTP saya sudah menggunakan KTP ...,Saat ini Allo Bank belum mendukung registrasi ...,54,111
165,LAYANAN,Bagaimana perhitungan bunga di Allo Prime?,Anda dapat melihat detail terkait suku bunga y...,42,123
211,LAYANAN,How much does it cost to save money in the All...,There are no administration fees to customers ...,53,76


In [ ]:
# Average character length per category
